<a href="https://colab.research.google.com/github/hc2twv/UPSE_OP/blob/main/OCHSimple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Código en Python que implemente el algoritmo de optimización de hormigas (ACO) para resolver el problema del vendedor viajero (TSP). Este es un problema clásico en el cual se busca encontrar la ruta más corta que visite una serie de ciudades y vuelva a la ciudad de origen.

In [7]:
import numpy as np

class AntColonyOptimizer:
    def __init__(self, distances, n_ants, n_best, n_iterations, decay, alpha=1, beta=1, start=None, end=None):
        """
        Inicializa el optimizador de colonia de hormigas.

        :param distances: Matriz de distancias entre ciudades.
        :param n_ants: Número de hormigas.
        :param n_best: Número de mejores hormigas para actualizar los rastros de feromonas.
        :param n_iterations: Número de iteraciones del algoritmo.
        :param decay: Tasa de evaporación de feromonas.
        :param alpha: Influencia de las feromonas en la probabilidad de selección del camino.
        :param beta: Influencia de la visibilidad (inverso de la distancia) en la probabilidad de selección del camino.
        :param start: Ciudad de inicio para todas las hormigas.
        :param end: Ciudad final para todas las hormigas.
        """
        self.distances = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_indices = range(len(distances))
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta
        self.start = start
        self.end = end

    def run(self):
        """
        Ejecuta el algoritmo ACO y devuelve la mejor ruta y su longitud.
        """
        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)
        for i in range(self.n_iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheromone(all_paths, self.n_best)
            shortest_path = min(all_paths, key=lambda x: x[1])
            if shortest_path[1] < all_time_shortest_path[1]:
                all_time_shortest_path = shortest_path
            self.pheromone *= self.decay
        return all_time_shortest_path

    def gen_path_dist(self, path):
        """
        Calcula la distancia total de una ruta.

        :param path: Lista de índices que representan una ruta.
        :return: Distancia total de la ruta.
        """
        total_dist = 0
        for ele in path:
            total_dist += self.distances[ele]
        return total_dist

    def gen_all_paths(self):
        """
        Genera todas las rutas para las hormigas.

        :return: Lista de todas las rutas y sus distancias.
        """
        all_paths = []
        for _ in range(self.n_ants):
            path = self.gen_path(self.start, self.end)
            all_paths.append((path, self.gen_path_dist(path)))
        return all_paths

    def gen_path(self, start, end):
        """
        Genera una ruta individual para una hormiga.

        :param start: Ciudad de inicio.
        :param end: Ciudad final.
        :return: Lista de índices que representan una ruta.
        """
        path = []
        visited = set()
        visited.add(start)
        prev = start
        for i in range(len(self.distances) - 1):
            move = self.pick_move(self.pheromone[prev], self.distances[prev], visited)
            if move == end:
                break
            path.append((prev, move))
            prev = move
            visited.add(move)
        path.append((prev, end))  # Añadir el punto final
        return path

    def spread_pheromone(self, all_paths, n_best):
        """
        Actualiza los rastros de feromonas basado en las mejores rutas.

        :param all_paths: Lista de todas las rutas y sus distancias.
        :param n_best: Número de mejores rutas a considerar para actualizar las feromonas.
        """
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, dist in sorted_paths[:n_best]:
            for move in path:
                self.pheromone[move] += 1.0 / dist

    def pick_move(self, pheromone, dist, visited):
        """
        Selecciona la próxima ciudad para visitar basada en las feromonas y la visibilidad.

        :param pheromone: Lista de valores de feromonas para las posibles ciudades.
        :param dist: Lista de distancias a las posibles ciudades.
        :param visited: Conjunto de ciudades ya visitadas.
        :return: Índice de la próxima ciudad a visitar.
        """
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0

        # Manejar distancias cero reemplazándolas con un valor pequeño
        dist = np.copy(dist)
        dist[dist == 0] = 1e-10

        row = pheromone ** self.alpha * ((1.0 / dist) ** self.beta)

        norm_row = row / row.sum()
        norm_row = np.nan_to_num(norm_row)  # Manejar NaN
        if norm_row.sum() == 0:
            norm_row = np.ones_like(norm_row) / len(norm_row)
        move = np.random.choice(self.all_indices, 1, p=norm_row)[0]
        return move

# Ejemplo de uso:
# Matriz de distancias (simétrica)
distances = np.array([
    [0, 2, 2, 5, 7],
    [2, 0, 4, 8, 2],
    [2, 4, 0, 1, 3],
    [5, 8, 1, 0, 2],
    [7, 2, 3, 2, 0]
])

# Inicializar el algoritmo
start_city = 0  # Ciudad de inicio
end_city = 4    # Ciudad final
aco = AntColonyOptimizer(distances, n_ants=10, n_best=3, n_iterations=100, decay=0.95, alpha=1, beta=2, start=start_city, end=end_city)

# Ejecutar el algoritmo
shortest_path = aco.run()
print(f"La ruta más corta encontrada es: {shortest_path[0]} con una distancia de {shortest_path[1]}")


<ipython-input-7-c2249fe685a2>:119: RuntimeWarning: divide by zero encountered in divide
  row = pheromone ** self.alpha * ((1.0 / dist) ** self.beta)
<ipython-input-7-c2249fe685a2>:119: RuntimeWarning: invalid value encountered in multiply
  row = pheromone ** self.alpha * ((1.0 / dist) ** self.beta)


La ruta más corta encontrada es: [(0, 1), (1, 4)] con una distancia de 4


El siguiente código en Python implementa un algoritmo de colonia de hormigas (ACO) para optimizar la colocación de nodos en una red de telecomunicaciones. El objetivo es minimizar la distancia total entre los nodos, mejorando la cobertura y la eficiencia de la red.

In [29]:
import numpy as np

class AntColonyTelecom:
    def __init__(self, distance_matrix, n_ants=10, n_iterations=100, alpha=1, beta=2, evaporation_rate=0.5):
        """
        Inicializa el algoritmo de Colonias de Hormigas para un problema de telecomunicaciones.

        :param distance_matrix: Matriz de distancias entre los nodos de la red.
        :param n_ants: Número de hormigas en la colonia.
        :param n_iterations: Número de iteraciones del algoritmo.
        :param alpha: Peso de la feromona en la elección de la siguiente ciudad.
        :param beta: Peso de la distancia en la elección de la siguiente ciudad.
        :param evaporation_rate: Tasa de evaporación de feromonas después de cada iteración.
        """
        self.distance_matrix = distance_matrix
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.pheromone_matrix = np.ones_like(distance_matrix) / np.mean(distance_matrix)  # Inicialización de feromonas

    def run(self, start_node, end_node):
        """
        Ejecuta el algoritmo ACO para encontrar la mejor ruta entre el nodo de inicio y el nodo de destino.

        :param start_node: Nodo de inicio.
        :param end_node: Nodo de destino.
        :return: La mejor ruta encontrada.
        """
        best_path = None
        best_distance = np.inf
        for _ in range(self.n_iterations):
            paths = self.generate_ant_paths(start_node, end_node)
            self.update_pheromones(paths)
            shortest_path, shortest_distance = min(paths, key=lambda x: x[1])
            if shortest_distance < best_distance:
                best_path = shortest_path
                best_distance = shortest_distance
        return best_path, best_distance

    def generate_ant_paths(self, start_node, end_node):
        """
        Genera las rutas de todas las hormigas.

        :param start_node: Nodo de inicio común para todas las hormigas.
        :param end_node: Nodo de destino común para todas las hormigas.
        :return: Lista de todas las rutas generadas por las hormigas.
        """
        paths = []
        for _ in range(self.n_ants):
            path = self.generate_ant_path(start_node, end_node)
            paths.append(path)
        return paths

    def generate_ant_path(self, start_node, end_node):
        """
        Genera la ruta de una hormiga.

        :param start_node: Nodo de inicio.
        :param end_node: Nodo de destino.
        :return: Ruta generada por la hormiga y su distancia total.
        """
        current_node = start_node
        path = [current_node]
        total_distance = 0
        while current_node != end_node:
            next_node = self.choose_next_node(current_node, end_node, path)
            total_distance += self.distance_matrix[current_node][next_node]
            path.append(next_node)
            current_node = next_node
        return path, total_distance

    def choose_next_node(self, current_node, end_node, path):
        """
        Elige el próximo nodo a visitar por una hormiga.

        :param current_node: Nodo actual.
        :param end_node: Nodo de destino.
        :param path: Ruta actual de la hormiga.
        :return: Próximo nodo a visitar.
        """
        pheromones = self.pheromone_matrix[current_node]
        distances = self.distance_matrix[current_node]
        valid_nodes = [node for node in range(len(pheromones)) if node not in path]
        probabilities = [self.calculate_probability(current_node, node, end_node, pheromones, distances) for node in valid_nodes]
        if all(prob == 0 for prob in probabilities):
            return np.random.choice(valid_nodes)  # Si todas las probabilidades son cero, elige aleatoriamente
        else:
            total_probability = sum(probabilities)
            normalized_probabilities = [prob / total_probability for prob in probabilities]
            return np.random.choice(valid_nodes, p=normalized_probabilities)

    def calculate_probability(self, current_node, next_node, end_node, pheromones, distances):
        """
        Calcula la probabilidad de moverse a un nodo específico.

        :param current_node: Nodo actual.
        :param next_node: Nodo al que se quiere mover.
        :param end_node: Nodo de destino.
        :param pheromones: Feromonas en los nodos vecinos.
        :param distances: Distancias a los nodos vecinos.
        :return: Probabilidad de moverse al nodo siguiente.
        """
        pheromone = pheromones[next_node]
        distance = distances[next_node]
        visibility = 1.0 / distance
        total_pheromone = sum((pheromone ** self.alpha) * visibility for pheromone in pheromones)
        probability = (pheromone ** self.alpha) * visibility / total_pheromone if total_pheromone > 0 else 0
        return probability

    def update_pheromones(self, paths):
        """
        Actualiza las feromonas en la matriz de feromonas basándose en las rutas de las hormigas.

        :param paths: Lista de rutas de las hormigas.
        """
        self.pheromone_matrix *= (1 - self.evaporation_rate)  # Evaporación de feromonas
        for path, distance in paths:
            pheromone_to_deposit = 1 / distance
            for i in range(len(path) - 1):
                current_node = path[i]
                next_node = path[i + 1]
                self.pheromone_matrix[current_node][next_node] += pheromone_to_deposit
                self.pheromone_matrix[next_node][current_node] += pheromone_to_deposit  # Asegurar simetría

    def calculate_path_distance(self, path):
        """
        Calcula la distancia total de una ruta dada.

        :param path: Ruta para la que se quiere calcular la distancia.
        :return: Distancia total de la ruta.
        """
        distance = sum(self.distance_matrix[path[i]][path[i + 1]] for i in range(len(path) - 1))
        return distance

# Ejemplo de uso:
# Creamos una matriz de distancias de ejemplo
distance_matrix = np.array([[0, 2, 3, 4],
                            [2, 0, 5, 6],
                            [3, 5, 0, 7],
                            [4, 6, 7, 0]])

# Creamos una instancia del algoritmo de Colonias de Hormigas
aco = AntColonyTelecom(distance_matrix)

# Definimos los nodos de inicio y fin
start_node = 0
end_node = 2

# Ejecutamos el algoritmo para encontrar la mejor ruta entre los nodos de inicio y fin
best_path, best_distance = aco.run(start_node, end_node)
print(f"La mejor ruta encontrada entre el nodo {start_node} y el nodo {end_node} es: {best_path}, con una distancia de {best_distance}.")



La mejor ruta encontrada entre el nodo 0 y el nodo 2 es: [0, 2], con una distancia de 3.


En este ejemplo, resolveremos el problema del Viajante de Comercio (TSP, Traveling Salesman Problem) utilizando el algoritmo de hormugas de ordenacion.

In [31]:
import numpy as np

class AntSystem:
    def __init__(self, distance_matrix, n_ants=10, n_iterations=100, alpha=1, beta=2, evaporation_rate=0.5):
        """
        Inicializa el algoritmo de Sistema de Hormigas (AS).

        :param distance_matrix: Matriz de distancias entre las ciudades.
        :param n_ants: Número de hormigas en la colonia.
        :param n_iterations: Número de iteraciones del algoritmo.
        :param alpha: Peso de la feromona en la elección de la siguiente ciudad.
        :param beta: Peso de la distancia en la elección de la siguiente ciudad.
        :param evaporation_rate: Tasa de evaporación de feromonas después de cada iteración.
        """
        self.distance_matrix = distance_matrix
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.pheromone_matrix = np.ones_like(distance_matrix) / np.mean(distance_matrix)  # Inicialización de feromonas

    def run(self):
        """
        Ejecuta el algoritmo AS para encontrar la mejor ruta entre las ciudades.

        :return: La mejor ruta encontrada y su distancia.
        """
        best_path = None
        best_distance = np.inf
        for _ in range(self.n_iterations):
            paths = self.generate_ant_paths()
            self.update_pheromones(paths)
            shortest_path, shortest_distance = min(paths, key=lambda x: x[1])
            if shortest_distance < best_distance:
                best_path = shortest_path
                best_distance = shortest_distance
        return best_path, best_distance

    def generate_ant_paths(self):
        """
        Genera las rutas de todas las hormigas.

        :return: Lista de todas las rutas generadas por las hormigas.
        """
        paths = []
        for _ in range(self.n_ants):
            path = self.generate_ant_path()
            paths.append(path)
        return paths

    def generate_ant_path(self):
        """
        Genera la ruta de una hormiga.

        :return: Ruta generada por la hormiga y su distancia total.
        """
        current_city = np.random.randint(len(self.distance_matrix))  # Seleccionamos una ciudad aleatoria como punto de partida
        unvisited_cities = set(range(len(self.distance_matrix)))
        unvisited_cities.remove(current_city)
        path = [current_city]
        total_distance = 0
        while unvisited_cities:
            next_city = self.choose_next_city(current_city, unvisited_cities)
            total_distance += self.distance_matrix[current_city][next_city]
            path.append(next_city)
            unvisited_cities.remove(next_city)
            current_city = next_city
        total_distance += self.distance_matrix[path[-1]][path[0]]  # Añadimos la distancia de vuelta a la ciudad inicial
        return path, total_distance

    def choose_next_city(self, current_city, unvisited_cities):
        """
        Elige la próxima ciudad a visitar por una hormiga.

        :param current_city: Ciudad actual.
        :param unvisited_cities: Conjunto de ciudades no visitadas.
        :return: Próxima ciudad a visitar.
        """
        probabilities = self.calculate_probabilities(current_city, unvisited_cities)
        next_city = np.random.choice(list(unvisited_cities), p=probabilities)
        return next_city

    def calculate_probabilities(self, current_city, unvisited_cities):
        """
        Calcula las probabilidades de visitar las ciudades no visitadas desde la ciudad actual.

        :param current_city: Ciudad actual.
        :param unvisited_cities: Conjunto de ciudades no visitadas.
        :return: Lista de probabilidades de visitar cada ciudad.
        """
        pheromones = self.pheromone_matrix[current_city, list(unvisited_cities)]
        distances = self.distance_matrix[current_city, list(unvisited_cities)]
        visibility = 1 / distances
        total_pheromone = np.sum(pheromones ** self.alpha * visibility ** self.beta)
        probabilities = (pheromones ** self.alpha * visibility ** self.beta) / total_pheromone
        return probabilities

    def update_pheromones(self, paths):
        """
        Actualiza las feromonas en la matriz de feromonas basándose en las rutas de las hormigas.

        :param paths: Lista de rutas de las hormigas.
        """
        self.pheromone_matrix *= (1 - self.evaporation_rate)  # Evaporación de feromonas
        for path, distance in paths:
            for i in range(len(path) - 1):
                self.pheromone_matrix[path[i], path[i+1]] += 1 / distance
                self.pheromone_matrix[path[i+1], path[i]] += 1 / distance  # Asegurar simetría


# Ejemplo de uso:
# Creamos una matriz de distancias de ejemplo
distance_matrix = np.array([[0, 10, 15, 20],
                            [10, 0, 35, 25],
                            [15, 35, 0, 30],
                            [20, 25, 30, 0]])

# Creamos una instancia del algoritmo AS
as_algorithm = AntSystem(distance_matrix)

# Ejecutamos el algoritmo para encontrar la mejor ruta
best_path, best_distance = as_algorithm.run()
print(f"La mejor ruta encontrada es: {best_path}, con una distancia total de {best_distance}.")


La mejor ruta encontrada es: [0, 2, 3, 1], con una distancia total de 80.
